# APDFT Analysis

Predictions can use

- **QC**: HF, CCSD, or CCSD(T) energy differences with the species of interests or target,
- **APDFT**: HF, CCSD, or CCSD(T) energy differences by changing the nuclear charge of other systems that have the same number of electrons as the target,
- **APDFT*n***: Taylor series approximation with order *n* of the APDFT potential energy surface.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

from qa_tools.utils import hartree_to_ev, all_atom_systems
from qa_tools.data import prepare_dfs
from qa_tools.prediction import get_qats_change_charge, get_qc_change_charge
from qa_tools.analysis import qats_error_change_charge

json_path = '../../json-data/atom-pyscf.qa-data.posthf.json'
df_qc, df_qats = prepare_dfs(json_path, get_CBS=False)

## APDFT prediction errors

There is some intrinsic error in modeling a target system (e.g., N atom) by changing the nuclear charge of a reference system's basis set (e.g., C<sup> &ndash;</sup> ).
The following cell computes this error, and represents the best performance for APDFT without fortuitous errors.

In [2]:
system_label = 'n'
delta_charge = -1
target_initial_charge = 0  # Initial charge of the system.
basis_set = 'aug-cc-pV5Z'  # cc-pV5Z, aug-cc-pVTZ, aug-cc-pVQZ, aug-cc-pV5Z, CBS-aug
change_signs = True  # Multiple all predictions by negative one (e.g., for electron affinities)

In [3]:
use_qats = False  # Use finite differences with Taylor series for APDFT predictions.

ea_qc_prediction = get_qc_change_charge(
    df_qc, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set
)
ea_qats_predictions = get_qats_change_charge(
    df_qc, df_qats, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set, use_qats=use_qats
)

ea_qc_prediction = hartree_to_ev(ea_qc_prediction)
ea_qats_predictions = {key:hartree_to_ev(value) for (key,value) in ea_qats_predictions.items()}
ea_qats_errors = {key:value-ea_qc_prediction for (key,value) in ea_qats_predictions.items()}

print(f'PySCF prediction of EA for {system_label}: {ea_qc_prediction:.3f} eV\n')
print(f'APDFT prediction errors in eV:')
print(pd.DataFrame(ea_qats_errors, index=[f'APDFT']))

PySCF prediction of EA for n: -0.201 eV

APDFT prediction errors in eV:
              c         f        o
APDFT  0.032448 -0.040251 -0.01219


## APDFT*n* prediction errors

Now, we can look at approximating the APDFT prediction by using a Taylor series centered on $\Delta Z = 0$.

In [4]:
use_qats = True  # Use finite differences with Taylor series for APDFT predictions.

ea_qc_prediction = get_qc_change_charge(
    df_qc, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set
)
ea_qats_predictions = get_qats_change_charge(
    df_qc, df_qats, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set, use_qats=use_qats
)

ea_qc_prediction = hartree_to_ev(ea_qc_prediction)
ea_qats_predictions = {key:hartree_to_ev(value) for (key,value) in ea_qats_predictions.items()}
ea_qats_errors = {key:value-ea_qc_prediction for (key,value) in ea_qats_predictions.items()}

print(f'PySCF prediction of EA for {system_label}: {ea_qc_prediction:.3f} eV\n')
print(f'APDFTn prediction errors in eV:')
print(pd.DataFrame(ea_qats_errors, index=[f'APDFT{n}' for n in range(5)]))

PySCF prediction of EA for n: -0.201 eV

APDFTn prediction errors in eV:
                 c          f          o
APDFT0   -4.848905  35.104146  13.771906
APDFT1   -1.813128 -14.690398  -3.919313
APDFT2   -3.834905  -0.591517  -0.214818
APDFT3   34.303740  -0.312760  -0.099927
APDFT4 -291.468413  -0.183423  -0.053606


### Specifying lambda values

We can also specify specific lambda values to include. For example, we could only look at lambda values of +-1.

In [5]:
considered_lambdas = [-1, 1]

use_qats = True  # Use finite differences with Taylor series for APDFT predictions.

ea_qc_prediction = get_qc_change_charge(
    df_qc, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set
)
ea_qats_predictions = get_qats_change_charge(
    df_qc, df_qats, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set, use_qats=use_qats,
    considered_lambdas=considered_lambdas
)

ea_qc_prediction = hartree_to_ev(ea_qc_prediction)
ea_qats_predictions = {key:hartree_to_ev(value) for (key,value) in ea_qats_predictions.items()}
ea_qats_errors = {key:value-ea_qc_prediction for (key,value) in ea_qats_predictions.items()}

print(f'PySCF prediction of EA for {system_label}: {ea_qc_prediction:.3f} eV\n')
print(f'APDFTn prediction errors in eV:')
print(pd.DataFrame(ea_qats_errors, index=[f'APDFT{n}' for n in range(5)]))

PySCF prediction of EA for n: -0.201 eV

APDFTn prediction errors in eV:
                 c          o
APDFT0   -4.848905  13.771906
APDFT1   -1.813128  -3.919313
APDFT2   -3.834905  -0.214818
APDFT3   34.303740  -0.099927
APDFT4 -291.468413  -0.053606


## APDFT*n* errors with respect to APDFT

Or you, can compute the difference between APDFT*n* (predictions with Taylor series) and APDFT.

In [6]:
compute_difference = True  # Returns APDFTn - APDFT instead of energy predictions.

use_qats = True  # Use finite differences with Taylor series for APDFT predictions.

ea_qats_predictions = get_qats_change_charge(
    df_qc, df_qats, system_label, delta_charge,
    target_initial_charge=target_initial_charge,
    change_signs=change_signs, basis_set=basis_set, use_qats=use_qats,
    compute_difference=compute_difference
)

ea_qats_predictions = {key:hartree_to_ev(value) for (key,value) in ea_qats_predictions.items()}

print(f'Differences between APDFTn and APDFT in eV:')
print(pd.DataFrame(ea_qats_predictions, index=[f'APDFT{n}' for n in range(5)]))

Differences between APDFTn and APDFT in eV:
                 c          f          o
APDFT0   -4.881353  35.144397  13.784096
APDFT1   -1.845575 -14.650147  -3.907123
APDFT2   -3.867353  -0.551266  -0.202628
APDFT3   34.271292  -0.272509  -0.087737
APDFT4 -291.500860  -0.143172  -0.041416


## Overall statistics

We can also compute mean absolute errors (MAEs), root mean squared error (RMSE) and max error.

In [7]:
all_systems = all_atom_systems[0:-1]
basis_set = 'aug-cc-pV5Z'
target_initial_charge = 0

use_qats = True
compute_difference = False
considered_lambdas = None

max_qa_order = 4
ignore_one_row = True

for i in range(len(all_systems)):
    sys_error = qats_error_change_charge(
        df_qc, df_qats, all_systems[i], delta_charge, change_signs=change_signs, 
        basis_set=basis_set, target_initial_charge=target_initial_charge,
        use_qats=use_qats, ignore_one_row=ignore_one_row,
        compute_difference=compute_difference,
        considered_lambdas=considered_lambdas
    )
    if i == 0:
        all_error = sys_error
    else:
        all_error = pd.concat(
            [all_error, sys_error], axis=1
        )

if use_qats or compute_difference == True:
    # MAE
    for n in range(0, max_qa_order+1):
        apdftn_errors = all_error.iloc[n].values
        apdftn_mae = np.mean(np.abs(apdftn_errors))
        print(f'APDFT{n} MAE: {apdftn_mae:.4f} eV')

    # RMSE
    print()
    for n in range(0, max_qa_order+1):
        apdftn_errors = all_error.iloc[n].values
        apdftn_rmse = np.sqrt(np.mean((apdftn_errors)**2))
        print(f'APDFT{n} RMSE: {apdftn_rmse:.4f} eV')
    
    # Max
    print()
    for n in range(0, max_qa_order+1):
        apdftn_errors = all_error.iloc[n].values
        apdftn_max = np.max(np.abs(apdftn_errors))
        print(f'APDFT{n} Max Abs.: {apdftn_max:.4f} eV')
else:
    # MAE
    apdftn_errors = all_error.iloc[0].values
    apdftn_mae = np.mean(np.abs(apdftn_errors))
    print(f'APDFT MAE: {apdftn_mae:.4f} eV')

    # RMSE
    print()
    apdftn_rmse = np.sqrt(np.mean((apdftn_errors)**2))
    print(f'APDFT RMSE: {apdftn_rmse:.4f} eV')

    # Max
    print()
    apdftn_max = np.max(np.abs(apdftn_errors))
    print(f'APDFT Max Abs.: {apdftn_max:.4f} eV')


APDFT0 MAE: 15.1554 eV
APDFT1 MAE: 6.2224 eV
APDFT2 MAE: 1.1140 eV
APDFT3 MAE: 1.6357 eV
APDFT4 MAE: 11.5901 eV

APDFT0 RMSE: 20.1437 eV
APDFT1 RMSE: 10.4108 eV
APDFT2 RMSE: 1.6244 eV
APDFT3 RMSE: 5.3115 eV
APDFT4 RMSE: 43.1342 eV

APDFT0 Max Abs.: 74.3534 eV
APDFT1 Max Abs.: 54.1903 eV
APDFT2 Max Abs.: 3.8349 eV
APDFT3 Max Abs.: 34.3037 eV
APDFT4 Max Abs.: 291.4684 eV
